<a href="https://colab.research.google.com/github/shantanu4u/DataQuality/blob/main/Dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install fuzzymatcher
!pip install recordlinkage
!pip install textdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage
from certifi import where
import psycopg2
import pandas.io.sql as psql
from sklearn.metrics import pair_confusion_matrix
import textdistance
import csv

## Example using fuzzymatcher

In [20]:
url_main = 'https://raw.githubusercontent.com/shantanu4u/DataQuality/main/Data/Customer_Main.csv'
datafr_1 = pd.read_csv(url_main)
datafr_1.head(3)

,Golden_Id,First_Name,Last_Name,Email,City,Street,Country,Currency_Code,DUNS,Address,Phone,Company,Customer_Classification,LBU,Market_Segment
0,1,Kenna,Dagwell,kdagwell0@tinyurl.com,Pigcawayan,Sunnyside,Philippines,PHP,35-751-8883,Tennyson,809-720-8464,Ntags,Honda,Granite,Waterproofer
1,2,Hailey,Adamowicz,hadamowicz1@bloglines.com,Stráž nad Nisou,Center,Czech Republic,CZK,48-582-2414,Jay,129-380-2548,Thoughtworks,Hyundai,Plastic,Painter
2,3,Cchaddie,Rayment,crayment2@hp.com,Yelizavetino,Shoshone,Russia,RUB,95-702-9369,Magdeline,859-712-0649,Vimbo,Chrysler,Vinyl,Plasterers


In [21]:
url_node = 'https://raw.githubusercontent.com/shantanu4u/DataQuality/main/Data/Customer_Main_Duplicate.csv'
datafr_2 = pd.read_csv(url_node)
datafr_2.head(3)  

,Id,First_Name,Last_Name,Email,City,Street,Country,Currency_Code,DUNS,Address,Phone,Company,Customer_Classification,LBU,Market_Segment
0,1,Kenna,Dagwell,kdagwell0@tinyurl.com,Pigcawayan,Sunnyside,Philippines,PHP,35-751-8883,Tennyson,809-720-8464,Ntags,Honda,Granite,Waterproofer
1,2,Hailey,Adamowicz,hadamowicz1@bloglines.co.tc,Stráž nad Nisou,Center,Czech Republic,CZK,48-582-2414,Jay,129-380-2548,Thoughtworks,Hyundai,Plastic,Painter
2,3,Cchaddie,Rayment,crayment2@hp.co.tc,Yelizavetino,Shoshone,Russia,RUB,95-702-9369,Magdeline,859-712-0649,Vimbo,Chrysler,Vinyl,Plasterers


In [22]:
datafr_2 = datafr_2.rename(columns={'First_Name': 'First_Name_2', 'Last_Name': 'Last_Name_2',  'Email': 'Email_2',  'City': 'City_2',  'Street': 'Street_2',
                                     'Country': 'Country_2', 'Address': 'Address_2', 'DUNS': 'DUNS_2', 'Company': 'Company_2', 'Customer_Classification': 'Customer_Classification_2',
                                    'LBU': 'LBU_2', 'Market_Segment': 'Market_Segment_2', 'Currency_Code': 'Currency_Code_2', 'Phone': 'Phone_2'})

In [23]:
#separated the data file 
    # datafr_1 = dataframTemp.sample(frac = 0.5)
    
    # # Creating dataframe with
    # # rest of the 50% values
    # datafr_2 = dataframTemp.drop(datafr_1.index)

    # datafr_2 = datafr_2.rename(columns={'First_Name': 'First_Name_2', 'Last_Name': 'Last_Name_2',  'Email': 'Email_2',  'City': 'City_2',  'Street': 'Street_2',
    #                                  'Country': 'Country_2', 'Address': 'Address_2', 'DUNS': 'DUNS_2', 'Company': 'Company_2', 'Gender': 'Gender_2'})

In [24]:
# Columns to match on from df_left
left_on = ["First_Name", "Last_Name", "Email", "City","Street","Country","Address","DUNS","Company","Currency_Code","Phone","Customer_Classification","LBU","Market_Segment"]

# Columns to match on from df_right
right_on = ["First_Name_2", "Last_Name_2", "Email_2", "City_2","Street_2","Country_2","Address_2","DUNS_2","Company_2","Customer_Classification_2","LBU_2","Market_Segment_2","Currency_Code_2","Phone_2",]

In [25]:

# Now perform the match
# It will take several minutes to run on this data set
matched_results = fuzzymatcher.fuzzy_left_join(datafr_1,
                                               datafr_2,
                                               left_on,
                                               right_on,
                                               left_id_col='Golden_Id',
                                               right_id_col='Id')

In [26]:
# Reorder the columns to make viewing easier
cols = [
    "best_match_score", "First_Name", "First_Name_2", "Last_Name", "Last_Name_2", "Email", "Email_2", "City", "City_2","Street", "Street_2","Country", "Country_2",
    "Address", "Address_2","DUNS", "DUNS_2","Company","Company_2","Customer_Classification","Customer_Classification_2","LBU","LBU_2","Market_Segment","Market_Segment_2"
]

In [27]:
# Let's see the best matches
matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,First_Name,First_Name_2,Last_Name,Last_Name_2,Email,Email_2,City,City_2,Street,...,DUNS,DUNS_2,Company,Company_2,Customer_Classification,Customer_Classification_2,LBU,LBU_2,Market_Segment,Market_Segment_2
4503,1.837769,Tisha,Tisha,Orgill,Orgill,torgilldc@hexun.com,torgilldc@hexun.com,Gif-sur-Yvette,Gif-sur-Yvette,Eagle Crest,...,68-862-6260,68-862-6260,Midel,Midel,Chevrolet,Chevrolet,Granite,Granite,Equipment Operator,Equipment Operator
1,1.758835,Hailey,Hailey,Adamowicz,Adamowicz,hadamowicz1@bloglines.com,hadamowicz1@bloglines.co.tc,Stráž nad Nisou,Stráž nad Nisou,Center,...,48-582-2414,48-582-2414,Thoughtworks,Thoughtworks,Hyundai,Hyundai,Plastic,Plastic,Painter,Painter
4549,1.716077,Meggie,Meggie,Januszewski,Januszewski,mjanuszewskiem@go.com,mjanuszewskiem@go.com,Phra Nakhon Si Ayutthaya,Phra Nakhon Si Ayutthaya,Becker,...,68-315-7575,68-315-7575,Photofeed,Photofeed,BMW,BMW,Glass,Glass,Cement Mason,Cement Mason
737,1.655464,Alyson,Alyson,Cunde,Cunde,acundei@so-net.ne.jp,acundei@so-net.ne.jp,Guang’an,Guang’an,Meadow Ridge,...,21-960-7838,21-960-7838,Mynte,Mynte,GMC,GMC,Stone,Stone,Laborer,Laborer
3341,1.653092,Ardisj,Ardisj,Kaveney,Kaveney,akaveney3x@ocn.ne.jp,akaveney3x@ocn.ne.jp,Kota Bharu,Kota Bharu,4th,...,27-579-8197,27-579-8197,Kazu,Kazu,Mazda,Mazda,Glass,Glass,Plumber,Plumber


In [28]:
# Now the worst matches
matched_results[cols].sort_values(by=['best_match_score'],
                                  ascending=True).head(5)

,best_match_score,First_Name,First_Name_2,Last_Name,Last_Name_2,Email,Email_2,City,City_2,Street,...,DUNS,DUNS_2,Company,Company_2,Customer_Classification,Customer_Classification_2,LBU,LBU_2,Market_Segment,Market_Segment_2
17587,-3.276003,Emma,Banky,De Filippi,Trevor,edefilippijz@sohu.com,bmichelleae@quantcast.com,San Juan de Limay,Chancay,Old Shore,...,11-752-4970,36-049-9618,Ailane,Kanoodle,Rolls-Royce,Lamborghini,Aluminum,Brass,Cement Mason,Brickmason
7603,-3.223187,Anatola,Adelheid,Orridge,Fishenden,aorridgega@newsvine.com,afishenden1g@nasa.gov,Nāḩiyat as Sab‘ Biyār,Nishishinminato,Lakeland,...,89-722-8219,65-722-8153,Dabtype,Ainyx,Hyundai,Nissan,Aluminum,Granite,Painter,Pipefitter
15735,-3.221166,Ulick,Gregorius,Baldoni,Wiley,ubaldonij9@github.io,gdabbotdoylea8@liveinternet.ru,Dowr-e Rabāţ,Mazār-e Sharīf,Butternut,...,45-719-2764,77-471-3638,Eare,Jabbersphere,Nissan,MINI,Aluminum,Rubber,Welder,Brickmason
26971,-3.181989,Giacinta,Camala,Juppe,Barukh,gjuppenl@e-recht24.de,cmacdonell9f@1und1.de,Tây Đằng,Mtimbira,Bonner,...,58-073-8327,58-654-6775,Flipstorm,Linkbuzz,Mitsubishi,Volkswagen,Glass,Wood,Pipelayer,Cement Mason
19451,-3.144527,Shaine,Mar,Fernie,Denzilow,sferniekp@nationalgeographic.com,mdenzilow1l@nationalgeographic.co.tc,San Francisco de Macorís,Hejiang,Coolidge,...,40-798-9106,87-625-0708,Talane,Leexo,Chevrolet,Pontiac,Stone,Steel,Pipefitter,HVAC


## Example using Python Record Linkage Toolkit

In [30]:
# Re-read in the data using the index_col
datafr_main = pd.read_csv(
    'https://raw.githubusercontent.com/shantanu4u/DataQuality/main/Data/Customer_Main.csv',
    index_col='Golden_Id'
)
datafr_node = pd.read_csv(
    'https://raw.githubusercontent.com/shantanu4u/DataQuality/main/Data/Customer_Main_Duplicate.csv',
    index_col='Id'
)

In [ ]:
# # Re-read in the data using the index_col
# datafr_main = pd.read_csv(
#     'https://raw.githubusercontent.com/shantanu4u/DataQuality/main/Data/Customer_Main.csv'
# )
# datafr_node = pd.read_csv(
#     'https://raw.githubusercontent.com/shantanu4u/DataQuality/main/Data/Customer_Main_Duplicate.csv'
# )

In [ ]:
# datafr_node.head()

In [31]:
datafr_node = datafr_node.rename(columns={'First_Name': 'First_Name_2', 'Last_Name': 'Last_Name_2',  'Email': 'Email_2',  'City': 'City_2',  'Street': 'Street_2',
                                     'Country': 'Country_2', 'Address': 'Address_2', 'DUNS': 'DUNS_2', 'Company': 'Company_2', 'Phone': 'Phone_2', 'Customer_Classification': 'Customer_Classification_2',
                                    'LBU': 'LBU_2', 'Market_Segment': 'Market_Segment_2', 'Currency_Code': 'Currency_Code_2'})

In [ ]:
# datafr_main.set_index("Golden_Id")
# datafr_node.set_index("Golden_Id_2")

In [32]:
# Build the indexer
indexer = recordlinkage.Index()
# Can use full or block
# indexer.full()
#indexer.block(left_on='State', right_on='Country')

# Use sortedneighbor as a good option if data is not clean
indexer.sortedneighbourhood(left_on='Country', right_on='Country_2')

<Index>

In [33]:
candidates = indexer.index(datafr_main, datafr_node)

In [34]:
# Let's see how many matches we want to do
print(len(candidates))

41039


In [35]:
compare = recordlinkage.Compare()
compare.exact('City', 'City_2', label='City')
compare.string('First_Name',
               'First_Name_2',
               threshold=0.85,
               label='First_Name')
compare.string('Last_Name',
               'Last_Name_2',
               method='jarowinkler',
               threshold=0.85,
               label='Last_Name')
compare.string('Email',
               'Email_2',
               method='jaro',
               threshold=0.85,
               label='Email')
# compare.string('City',
#                'City_2',
#                method='levenshtein',
#                threshold=0.85,
#                label='City')
compare.string('Street',
               'Street_2',
               method='damerau_levenshtein',
               threshold=0.85,
               label='Street')
compare.string('Country',
               'Country_2',
               method='qgram',
               threshold=0.85,
               label='Country')
compare.string('Address',
               'Address_2',
               method='cosine',
               threshold=0.85,
               label='Address')
compare.string('Company',
               'Company_2',
               method='damerau_levenshtein',
               threshold=0.85,
               label='Last_Name')
compare.string('Customer_Classification',
               'Customer_Classification_2',
               method='jarowinkler',
               threshold=0.85,
               label='Customer_Classification')
compare.string('LBU',
               'LBU_2',
               method='levenshtein',
               threshold=0.85,
               label='LBU')
compare.string('Market_Segment',
               'Market_Segment_2',
               method='jarowinkler',
               threshold=0.85,
               label='Market_Segment')
features = compare.compute(candidates, datafr_main,
                           datafr_node)

In [36]:
features

City  First_Name  Last_Name  Email  Street  Country  Address  \
Golden_Id Id                                                                  
1         15      0         0.0        0.0    0.0     0.0      0.0      0.0   
          39      0         0.0        0.0    0.0     0.0      0.0      0.0   
          113     0         0.0        0.0    0.0     0.0      0.0      0.0   
          123     0         0.0        0.0    0.0     0.0      0.0      0.0   
          180     0         0.0        0.0    0.0     0.0      0.0      0.0   
...             ...         ...        ...    ...     ...      ...      ...   
877       523     0         0.0        0.0    0.0     0.0      0.0      0.0   
979       214     0         0.0        0.0    0.0     0.0      0.0      0.0   
          523     0         0.0        0.0    0.0     0.0      0.0      0.0   
981       214     0         0.0        0.0    0.0     0.0      0.0      0.0   
          523     0         0.0        0.0    0.0     0.0      0.0      0.0   

               Last_Name  Customer_Classification  LBU  Market_Segment  
Golden_Id Id                                                            
1         15         0.0                      0.0  1.0             0.0  
          39         0.0                      0.0  0.0             0.0  
          113        0.0                      0.0  0.0             0.0  
          123        0.0                      0.0  1.0             0.0  
          180        0.0                      0.0  0.0             0.0  
...                  ...                      ...  ...             ...  
877       523        0.0                      0.0  0.0             0.0  
979       214        0.0                      0.0  0.0             0.0  
          523        0.0                      0.0  1.0             0.0  
981       214        0.0                      0.0  0.0             0.0  
          523        0.0                      1.0  0.0             0.0  

[41039 rows x 11 columns]

In [37]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

11.0      340
10.0       90
9.0        99
4.0         5
3.0       277
2.0      4733
1.0     26625
0.0      8870
dtype: int64

Now we know that there are 8870 rows with no matching values whatsoever. 26618 rows have at least one match, 4712 have 2 and 297 have 3 matches.

To make the rest of the analysis easier, let’s get all the records with 2 or 3 matches and add a total score:

In [38]:
potential_matches = features[features.sum(axis=1) > 2].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'City':'Market_Segment'].sum(axis=1)
potential_matches.head(100)

,Golden_Id,Id,City,First_Name,Last_Name,Email,Street,Country,Address,Last_Name,Customer_Classification,LBU,Market_Segment,Score
0,890,80,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0
1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0
2,44,44,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0
3,46,46,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0
4,77,77,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,425,425,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0
96,447,447,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0
97,454,454,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0
98,465,465,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0


In [41]:
datafr_main.loc[425,:]



First_Name                            Hilario
Last_Name                           Etheredge
Email                      hetheredgebs@51.la
City                            Nizhniy Lomov
Street                                  Toban
Country                                Russia
Currency_Code                             RUB
DUNS                              70-407-0497
Address                                Hintze
Phone                            877-601-1395
Company                                 Twiyo
Customer_Classification             Chevrolet
LBU                                   Granite
Market_Segment             Sheet Metal Worker
Name: 425, dtype: object

In [40]:
datafr_node.loc[425,:]

First_Name_2                            Hilario
Last_Name_2                           Etheredge
Email_2                      hetheredgebs@51.la
City_2                            Nizhniy Lomov
Street_2                                  Toban
Country_2                                Russia
Currency_Code_2                             RUB
DUNS_2                              70-407-0497
Address_2                                Hintze
Phone_2                            877-601-1395
Company_2                                 Twiyo
Customer_Classification_2             Chevrolet
LBU_2                                   Granite
Market_Segment_2             Sheet Metal Worker
Name: 425, dtype: object

### Now that we know the matches, we need to wrangle the data to make it easier to review all the data together. I am going to make a concatenated name and address lookup for each of these source DataFrames.

In [42]:
datafr_main['Name_Address_Lookup'] = datafr_main[[
    'First_Name', 'Last_Name', 'City', 'Street','Country','Address'
]].apply(lambda x: '_'.join(x), axis=1)

datafr_node['Name_Address_2_Lookup'] = datafr_node[[
    'First_Name_2', 'Last_Name_2', 'City_2',
    'Street_2','Country_2','Address_2'
]].apply(lambda x: '_'.join(x), axis=1)

main_lookup = datafr_main[['Name_Address_Lookup']].reset_index()
node_lookup = datafr_node[['Name_Address_2_Lookup']].reset_index()

In [43]:
main_lookup.head()

,Golden_Id,Name_Address_Lookup
0,1,Kenna_Dagwell_Pigcawayan_Sunnyside_Philippines...
1,2,Hailey_Adamowicz_Stráž nad Nisou_Center_Czech ...
2,3,Cchaddie_Rayment_Yelizavetino_Shoshone_Russia_...
3,4,Skip_MacAulay_Xiashitai_Vera_China_Hanover
4,5,Jarad_Jardin_Yangzi_Kedzie_China_Linden


In [44]:
node_lookup.head()

,Id,Name_Address_2_Lookup
0,1,Kenna_Dagwell_Pigcawayan_Sunnyside_Philippines...
1,2,Hailey_Adamowicz_Stráž nad Nisou_Center_Czech ...
2,3,Cchaddie_Rayment_Yelizavetino_Shoshone_Russia_...
3,4,Skip_MacAulay_Xiashitai_Vera_CHN_Hanover
4,5,Jarad_Jardin_Yangzi_Kedzie_CHN_Linden


In [45]:
main_merge = potential_matches.merge(main_lookup, how='left')

In [46]:
final_merge = main_merge.merge(node_lookup, how='left')

In [47]:
cols = ['Golden_Id', 'Id', 'Score',
        'Name_Address_Lookup', 'Name_Address_2_Lookup']
final_merge[cols].sort_values(by=['Golden_Id', 'Score'], ascending=False)

,Golden_Id,Id,Score,Name_Address_Lookup,Name_Address_2_Lookup
465,998,385,3.0,Alfredo_Instone_Vetlanda_Farwell_Sweden_Scott,Karyn_Clinton_Arlöv_Troy_Sweden_Anhalt
352,997,217,3.0,Rori_Gabbitas_Zhetian_Derek_China_Quincy,Bartholomew_Boxer_Longquan_Huxley_China_Loomis
41,996,274,3.0,Haskell_Mathy_Bilad_Warner_Philippines_Fair Oaks,Noellyn_Bolstridge_Madalum_Hallows_Philippines...
351,993,464,3.0,Iolande_Screach_Dancheng_Loeprich_China_Northport,Regan_Liddle_Penghua_Fuller_China_Hallows
384,990,232,3.0,Aubert_Pracy_Carvalhal_Fisk_Portugal_Ronald Regan,Rivkah_Whardley_Tojeira_Stang_Portugal_Haas
...,...,...,...,...,...
385,7,7,11.0,Nickey_Boot_Saint-Paul_Riverside_Reunion_Ramsey,Nickey_Boot_Saint-Paul_Riverside_Reunion_Ramsey
353,6,6,11.0,Garrik_Pearne_Vale da Bajouca_Harper_Portugal_...,Garrik_Pearne_Vale da Bajouca_Harper_Portugal_...
49,3,3,11.0,Cchaddie_Rayment_Yelizavetino_Shoshone_Russia_...,Cchaddie_Rayment_Yelizavetino_Shoshone_Russia_...
42,2,2,11.0,Hailey_Adamowicz_Stráž nad Nisou_Center_Czech ...,Hailey_Adamowicz_Stráž nad Nisou_Center_Czech ...
